In [129]:
import pandas as pd

gs_res = pd.read_csv('output/lgb_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_res.head()

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,cv,feature_fraction,feature_fraction_seed,gs_timespent,learning_rate,max_depth,metric,min_data_in_leaf,num_leaves,num_threads,objective,pred_timespent,scale_pos_weight,timestamp,val_auc
5BJs,0.5,1,23577416,21,gbdt,True,1.0,23577416,0:00:00,0.1,-1,auc,28,15,8,binary,0:00:00,1,2018-07-03 20:48:30,0.704369


In [130]:
import pandas as pd

gs_nn_res = pd.read_csv('output/nn_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_nn_res.head()

,batch_size,best_epoch,cat_emb_outdim,combined_dense_n_layers,dense_units,drop_rate,ep_for_lr,gs_timespent,lr_fin,lr_init,max_ep,mode,monitor,nn_seed,num_dense_n_layers,patience,pred_batch_size,pred_timespent,timestamp,trn_acc,trn_loss,val_acc,val_auc,val_loss
kV7L,128,1,50,1,50,0.2,1,0:00:03,0.005,0.01,1,max,val_auc,24119099,1,1,50000,0:00:00,2018-07-03 20:48:31,0.7999,0.490096,0.8133,0.698756,0.442639


In [133]:
from os import listdir

for filename in listdir('output'):
    if '_grid_search' in filename:
        print(filename)

logreg_grid_search.csv
nn_grid_search.csv
lgb_grid_search.csv


In [132]:
stacknet_res = pd.read_csv('output/oof/stacknet_report.csv').sort_values(by=['oof_cv_score'])
stacknet_res

,model_data,chosen model_data,oof_cv_score,test_score,gs_val_auc
3,kV7L__NN_flight_data_ordinal_layer1,NaN,0.698561,0.715538,0.698756
2,iS9u__NN_layer2,5BJs__LGB,0.700628,0.713841,NaN
1,YqXH__LOGREG_layer2,5BJs__LGB,0.700630,0.713841,NaN
0,5BJs__LGB_flight_data_ordinal_layer1,NaN,0.700790,0.708197,0.704369


In [ ]:
# 1. oof_cv_score vs. gs_val_auc
# for NN, oof_cv_score != gs_val_auc is normal, because gs_val_auc is get by train+val in grid search, while 
# oof_cv_score is got by nfolds cv
# for LGB, if gs_cv is true in config file, then oof_cv_score == gs_val_auc (almost). otherwise, based on the same
# reason as NN, oof_cv_score != gs_val_auc is normal 

# 2. test_score
# for NN: the one(not showing in log) is got by train+val and load_weights when doing prediction after grid search
# (when do_preds is true in config file), and test_score is got during oof where all trainig data are used. It's 
# normal that they are not equal
# for LGB, since model is retrain using all training data when doing prediction after grid search, the one is not in 
# log == test_score

In [2]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [36]:
from automl_libs import BaseLayerResultsRepo

In [37]:
repo = BaseLayerResultsRepo(['label'], 'output/oof/', load_from_file=True)

In [38]:
for m,s in repo.show_scores():
    print(s,m)

0.7315492716625718 1530412261__LOGREG_layer2
0.7315275954635782 1530412229__LOGREG_layer2
0.7269551244985933 1530409216__LGB_flight_data_ordinal
0.7226665566391379 1530409210__LGB_flight_data_ordinal
0.7142048521932204 1530409222__LGB_flight_data_ordinal
0 1530410638__LOGREG_layer2


In [97]:
gs_res.head(1).T.to_dict()

{1530450414: {'bagging_fraction': 0.7,
  'bagging_freq': 1,
  'bagging_seed': 36256925,
  'best_round': 30,
  'boosting': 'gbdt',
  'cv': True,
  'feature_fraction': 1.0,
  'feature_fraction_seed': 36256925,
  'gs_timespent': '0:00:02',
  'learning_rate': 0.1,
  'max_depth': -1,
  'metric': 'auc',
  'min_data_in_leaf': 22,
  'num_leaves': 31,
  'num_threads': 8,
  'objective': 'binary',
  'pred_timespent': '0:00:01',
  'scale_pos_weight': 1,
  'timestamp': '2018-07-01 09:06:54',
  'val_auc': 0.737820625997909}}

In [134]:
from sklearn.svm import SVC

In [184]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [135]:
from sklearn.ensemble import BaggingClassifier

In [185]:
lsvc = CalibratedClassifierCV(LinearSVC())

In [187]:
lsvc.fit(np.array([0,1,2,3,4,5]).reshape(-1,1), [1,0,0,1,1,0])

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=3, method='sigmoid')

In [188]:
lsvc.predict_proba(2.22)

array([[0.53194697, 0.46805303]])

In [153]:
from sklearn.linear_model import LogisticRegression

In [176]:
svc = SVC(kernel='linear', probability=True)#, class_weight='auto')


In [177]:
svc.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [178]:
svc.predict_proba(2.22)

array([[0.42309385, 0.57690615]])

In [181]:
from sklearn.multiclass import OneVsRestClassifier

In [182]:
clf = OneVsRestClassifier(BaggingClassifier(svc, max_samples=1.0 / 5, n_estimators=5))
clf.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

OneVsRestClassifier(estimator=BaggingClassifier(base_estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.2, n_estimators=5, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
          n_jobs=1)

In [183]:
clf.predict_proba(2.22)

NotFittedError: predict_proba is not available when fitted with probability=False